# Experiment: Paddle OCR
[PaddleOCr](https://github.com/PaddlePaddle/PaddleOCR) belong to PaddlePaddle ecosystem, which is a quite nice library. 

PaddleOCR require is quite outdated environment (not suitable with Apple Silicon), and the performance (maybe) is optimized for Chinese. This notebooks its power on English.

Used this requirement (not sure it's correct :D) to install
```
loguru==0.7.2
setuptools==68.2.2
tqdm==4.66.1
typing_extensions==4.8.0
azure-ai-formrecognizer
azure-storage-blob
paddleocr==2.7.0.3
paddlepaddle==2.5.1
```


In [ ]:
import io
from PIL import Image
import fitz
import os
# import cv2
from paddleocr import PPStructure, draw_structure_result, save_structure_res
doc_file = "../dataset/rhb_dataset/openai_policy_search/01. S07_SM773_MY.V2.0.pdf"
cache_dir = "../dataset/rhb_dataset/openai_policy_search/01. S07_SM773_MY.V2.0.pdf-cache"
analyze_path = f"{cache_dir}/analyzed-result.json"
table_engine = PPStructure(show_log=True, lang='en', recovery=True)

page_num = 10
doc = fitz.open(doc_file) # open a document
page = doc[page_num]

In [ ]:
import numpy as np
import cv2
pil_image = Image.open(
    io.BytesIO(page.get_pixmap().pil_tobytes(format="PNG"))
)
pil_image.save("image.png")

image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

In [ ]:
from paddleocr.ppstructure.recovery.recovery_to_doc import sorted_layout_boxes, convert_info_docx
h, w, _ = image.shape
image_name = "test_image/image.png"

np.int = np.int32
result = table_engine(image)
save_structure_res(result, os.path.dirname(image_name), os.path.basename(image_name).split(".")[0])
for line in result:
    line.pop('img')
    print(line)

res = sorted_layout_boxes(result, w)
convert_info_docx(image, res, os.path.dirname(image_name), img_name=os.path.basename(image_name).split(".")[0])


font_path = './simfang.ttf'
im_show = draw_structure_result(image, result, font_path=font_path)
im_show = Image.fromarray(im_show)
display(im_show)


In [ ]:
from tqdm import tqdm


np.int = np.int32
FONT_PATH = './simfang.ttf'

def page2img(page):
    pil_image = Image.open(
        io.BytesIO(page.get_pixmap().pil_tobytes(format="PNG"))
    )
    pil_image.save("image.png")

    image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    return image

def run_extraction(doc_file:str, page_range):
    results = []
    table_engine = PPStructure(lang='en', recovery=True)
    doc = fitz.open(doc_file) # open a document
    pages = [doc[idx] for idx in range(*page_range)]
    for idx, page in tqdm(enumerate(pages), desc="Extraction..."):
        # Get data
        image = page2img(page)
        page_num = idx + page_range[0]
        h, w, _ = image.shape
        image_path = f"test_image/image-{page_num}.png"
        image_name = f"image-{page_num}.png"
        image_dir = f"test_image/"
        # OCR
    
        result = table_engine(image)
        save_structure_res(result, image_dir, image_name.split(".")[0])
        
        for line in result:
            line.pop('img')

        res = sorted_layout_boxes(result, w)
        convert_info_docx(
            image, 
            res, 
            image_dir, 
            img_name=image_name.split(".")[0]
        )

        im_show = draw_structure_result(image, result, font_path=FONT_PATH)
        im_show = Image.fromarray(im_show)
        results.append({
            'im_show': im_show,
            'result': result,
        })
    pass

data = run_extraction(doc_file, (10, 20))